<a href="https://colab.research.google.com/github/castorini/anserini-notebooks/blob/master/pyserini_msmarco_passage_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pyserini Demo on the MS MARCO Passage Dataset

This notebook replicates the BM25 baseline for the [MS MARCO passage ranking task](http://www.msmarco.org/) with [Pyserini](http://pyserini.io/), the Python interface to [Anserini](http://anserini.io).


## Installation


Install Python dependencies:


In [ ]:
%%capture
!pip install pyserini==0.10.0.1

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

## Usage

You can use `pysearch` to search over an index. The questions (called "topics" in TREC parlance) are already distributed in Pyserini:

In [ ]:
from pyserini.search import get_topics

topics = get_topics('msmarco_passage_dev_subset')
print(f'{len(topics)} queries total')

6980 queries total


Let's take a look at a specific question. Topics often have different "fields": "title" is the one we want. (Again, this is just TREC parlance.)

In [ ]:
topics[1102400]['title']

'why do bears hibernate'

Next, we can initialize a searcher with a pre-built index, which Pyserini will automatically download:

In [ ]:
from pyserini.search import SimpleSearcher

searcher = SimpleSearcher.from_prebuilt_index('msmarco-passage')

index-msmarco-passage-20201117-f87c94.tar.gz: 0.00B [00:00, ?B/s]

Attempting to initialize pre-built index msmarco-passage.


index-msmarco-passage-20201117-f87c94.tar.gz: 2.07GB [01:46, 20.8MB/s]                            


Extracting /root/.cache/pyserini/indexes/index-msmarco-passage-20201117-f87c94.tar.gz into /root/.cache/pyserini/indexes/index-msmarco-passage-20201117-f87c94.1efad4f1ae6a77e235042eff4be1612d...
Initializing msmarco-passage...


Now we can search:

In [ ]:
import json

hits = searcher.search(topics[1102400]['title'])

# Prints the first 10 hits
for i in range(0, 10):
    jsondoc = json.loads(hits[i].raw)
    print(f'{i+1:2} {hits[i].score:.5f} {jsondoc["contents"][:80]}...')

 1 17.33580 Why do Bears hibernate? March 31, 2010, Joan, Leave a comment. Why do bears hibe...
 2 13.23090 Why do bears hibernate? Watch this to discover how much effort is spent on survi...
 3 13.13570 Technically, as the other anwerer said, bears do not hibernate, but there isn't ...
 4 13.01460 It is a common misconception that bears hibernate during the winter. While bears...
 5 13.00390 To prepare for hibernation, grizzlies must prepare a den, and consume an immense...
 6 12.68940 Some zoo bears are fed year round, and do not hibernate. Since they do not under...
 7 12.55450 Bears in zoos will not hibernate if food is available, though they will slow dow...
 8 12.51710 All kinds of bears technically don't hibernate. They enter into a phase called t...
 9 12.43500 Date: 12-11-2012. It is a common misconception that bears hibernate during the w...
10 12.37460 While bears tend to slow down during the winter, they are not true hibernators. ...


The `hits` data structure holds the `docid`, the retrieval score, as well as the document content:

In [ ]:
from IPython.core.display import display, HTML
display(HTML('<div style="font-family: Times New Roman; padding-bottom:10px">' + hits[0].raw + '</div>'))

Let's run all the queries from the dev set:

In [ ]:
from pyserini.search import SimpleSearcher

def run_all_queries(file, topics, searcher):
    with open(file, 'w') as runfile:
        cnt = 0
        print('Running {} queries in total'.format(len(topics)))
        for id in topics:
            query = topics[id]['title']
            hits = searcher.search(query, 1000)
            for i in range(0, len(hits)):
                _ = runfile.write('{} Q0 {} {} {:.6f} Anserini\n'.format(id, hits[i].docid, i+1, hits[i].score))
            cnt += 1
            if cnt % 100 == 0:
                print(f'{cnt} queries completed')

searcher = SimpleSearcher.from_prebuilt_index('msmarco-passage')

run_all_queries('run-msmarco-passage-bm25.txt', topics, searcher)


Attempting to initialize pre-built index msmarco-passage.
/root/.cache/pyserini/indexes/index-msmarco-passage-20201117-f87c94.1efad4f1ae6a77e235042eff4be1612d already exists, skipping download.
Initializing msmarco-passage...
Running 6980 queries in total
100 queries completed
200 queries completed
300 queries completed
400 queries completed
500 queries completed
600 queries completed
700 queries completed
800 queries completed
900 queries completed
1000 queries completed
1100 queries completed
1200 queries completed
1300 queries completed
1400 queries completed
1500 queries completed
1600 queries completed
1700 queries completed
1800 queries completed
1900 queries completed
2000 queries completed
2100 queries completed
2200 queries completed
2300 queries completed
2400 queries completed
2500 queries completed
2600 queries completed
2700 queries completed
2800 queries completed
2900 queries completed
3000 queries completed
3100 queries completed
3200 queries completed
3300 queries comp

Let's evaluate using `trec_eval`: the expected MAP should be 0.1926.

In [ ]:
%%capture
!wget -O jtreceval-0.0.5-jar-with-dependencies.jar https://search.maven.org/remotecontent?filepath=uk/ac/gla/dcs/terrierteam/jtreceval/0.0.5/jtreceval-0.0.5-jar-with-dependencies.jar
!wget https://raw.githubusercontent.com/castorini/anserini/master/src/main/resources/topics-and-qrels/qrels.msmarco-passage.dev-subset.txt

In [ ]:
!java -jar jtreceval-0.0.5-jar-with-dependencies.jar qrels.msmarco-passage.dev-subset.txt run-msmarco-passage-bm25.txt

runid                 	all	Anserini
num_q                 	all	6980
num_ret               	all	6974598
num_rel               	all	7437
num_rel_ret           	all	6309
map                   	all	0.1926
gm_map                	all	0.0168
Rprec                 	all	0.1048
bpref                 	all	0.8526
recip_rank            	all	0.1960
iprec_at_recall_0.00  	all	0.1964
iprec_at_recall_0.10  	all	0.1964
iprec_at_recall_0.20  	all	0.1964
iprec_at_recall_0.30  	all	0.1964
iprec_at_recall_0.40  	all	0.1952
iprec_at_recall_0.50  	all	0.1952
iprec_at_recall_0.60  	all	0.1898
iprec_at_recall_0.70  	all	0.1898
iprec_at_recall_0.80  	all	0.1893
iprec_at_recall_0.90  	all	0.1893
iprec_at_recall_1.00  	all	0.1893
P_5                   	all	0.0591
P_10                  	all	0.0394
P_15                  	all	0.0301
P_20                  	all	0.0246
P_30                  	all	0.0182
P_100                 	all	0.0069
P_200                 	all	0.0038
P_500                 	all	0.0017
P_1000           